In [12]:
from tensorflow.keras.models import load_model
import levy
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
def estimate_all_params_2(X, beta=None):
    
#     X = (X - X.mean())/X.std()

    params = dict()
#     params["mu"], params['sigma'] = 0., 1.
    if beta is not None: 
        params["beta"] = beta
    
    params, neglog_density = levy.fit_levy(X)
    p = params.__dict__
    r = dict(zip(p["pnames"], p["_x"]))
    r["log_density"] = -neglog_density
    # print(r)
    return r
import matplotlib.pyplot as plt
import numpy as np

def compute_sparsity(weight_values):
    total_params = np.prod(weight_values.shape)  # 总参数数量
    zero_params = np.sum(abs(weight_values) < 0.01)  # 零参数数量
    # print(zero_params)
    sparsity = zero_params / total_params  # 稀疏度

    return sparsity

for depth, width in [(3, 3)]:
    print(f'depth={depth}, width={width}')
    if depth != 3:
        lr = 0.0005
        iters = 50000
    else:
        lr = 0.001
        iters = 10000
    model = Sequential()
    for j in range(depth-2):
        model.add(Dense(width, activation='relu',input_shape=(1,784)))
    model.add(Dense(10, activation='softmax'))

# model = lstm_self(input_shape,nb_class,num_units,num_layers)
model.load_weights('/home/xueqiongyuan/new_noise_code/models/libras_LSTM_width128_depth1_b32_alpha2.0_sigma0.2_num3_lr0.001_iter10000.h5')
sparsity = []
for layer in model.layers:
    for weight in layer.weights:
        # if 'bias' not in weight.name and 'batch_normalization' not in weight.name:
        if 'dense' in weight.name:  
            weight_values = weight.numpy()
            # print(weight.name)
            # plt.hist(weight_values.flatten(),bins = 50)
            # plt.title(f"Weight Histogram - {weight.name}")
            # plt.show()
            # print(estimate_all_params_2(weight_values.flatten()[:10000]))
            sparsity.append(compute_sparsity(weight_values))

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 45, 2)]           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               67072     
_________________________________________________________________
dense_5 (Dense)              (None, 15)                1935      
Total params: 69,007
Trainable params: 69,007
Non-trainable params: 0
_________________________________________________________________


In [13]:
np.mean(sparsity)

0.11015625000000001

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.layers import MaxPooling1D, Conv1D
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling1D, Permute, concatenate, Activation, add
import numpy as np
import math
input_shape = (45, 2)
nb_class = 15
num_units = 128
num_layers = 1
def lstm_self(input_shape, nb_class, num_units, num_layers):
    # Original proposal:
    # S. Hochreiter and J. Schmidhuber, “Long Short-Term Memory,” Neural Computation, vol. 9, no. 8, pp. 1735–1780, Nov. 1997.
        
    ip = Input(shape=input_shape)

    if num_layers == 1:
        l2 = LSTM(num_units)(ip)
    elif num_layers == 2:
        l1 = LSTM(num_units, return_sequences=True)(ip)
        l2 = LSTM(num_units)(l1)
    elif num_layers == 3:
        l0 = LSTM(num_units, return_sequences=True)(ip)
        l1 = LSTM(num_units, return_sequences=True)(l0)
        l2 = LSTM(num_units)(l1)

    out = Dense(nb_class, activation='softmax')(l2)

    model = Model([ip], [out])

    model.summary()

    return model